In [24]:
# Import required libraries
import numpy as np
import pandas as pd
import io
import requests
import random
import matplotlib.pyplot as plt

# Get data from online CSV file
url="https://raw.githubusercontent.com/elinlutz/gatsby-map/master/src/data/time_series/time_series_deaths-deaths.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

# Drop unecessary columns
c.drop(['Region', 'Population', 'Lat', 'Long', 'Today', 'Region_Deaths', 'FHM_Deaths_Today', 'Diff'], axis=1, inplace=True)

# Drop the last 3 rows
c = c.iloc[:-3]

# Traspose table
c = c.transpose()

# Make first row (region names) into header
new_header = c.iloc[0] # First row
c = c[1:] # All rows except header
c.columns = new_header # Set header row as header

# Remove all columns with only zeros
c.loc[:, (c != 0).any(axis=0)]

# Create index based on number of rows
c['Index'] = np.arange(len(c))
c = c.set_index('Index')

# Take 7 day rolling average
c = c.rolling(7).mean()

# Create row indicating first occurence of >=3 deaths in a day (rolling 7-day average)
e = c.ge(3).idxmax()
e = e.reset_index()
e['Index'] = np.arange(len(e))
e = e.set_index('Index')
c = c.transpose()
c = c.reset_index()
c['Index'] = np.arange(len(c))
c = c.set_index('Index')
c['First_Occ'] = e[0]
c = c.transpose()

# Make first row (region names) into header
new_header = c.iloc[0] #grab the first row for the header
c = c[1:] #take the data less the header row
c.columns = new_header #set the header row as the df header

# Only keep regions with a start day (an occurence of average > 3 deaths)
c = c.transpose()
is_start_day = c['First_Occ'] > 0
c_true = c[is_start_day]
c_true = c_true.transpose()
c_final = c_true.iloc[:-1,:]

# LOOP to shift values to start position
i_start = 0
i_end = 1

for col in c_final: 
    
    i_first_occ = c_true.iat[len(c_final), i_start]
    c_final.iloc[:,i_start:i_end] = c_final.iloc[:,i_start:i_end].shift(-i_first_occ, axis = 0)
    i_start = i_start + 1
    i_end = i_end + 1

# Remove N/A values
c_final = c_final.dropna(axis=0, how='all')

# Create lines indicating different doubling times
c_final['x'] = np.arange(len(c_final))
c_final['3-days'] = 3 * (2 ** 0.333333333 ) ** c_final['x']
c_final['7-days'] = 3 * (2 ** 0.142857143 ) ** c_final['x']
c_final['28-days'] = 3 * (2 ** 0.035714286 ) ** c_final['x']

c_final.drop('x', axis = 1, inplace=True)

# 
cols = list(c_final.columns.values) # List all of the columns in dataframe
cols.pop(cols.index('3-days')) # Remove b from list
cols.pop(cols.index('7-days')) # Remove x from list
cols.pop(cols.index('28-days')) #Remove x from list
c_final = c_final[['3-days','7-days', '28-days'] + cols] #Create new dataframe with columns in the order you want

# Update CSV file
c_final.to_csv('test.csv', encoding='utf-8-sig')
c_final


c:\python27\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Display_Name,3-days,7-days,28-days,Dalarna,Gävleborg,Skåne,Stockholm,Sörmland,Uppsala,Västmanland,Västra Götaland,Örebro,Östergötland
Index,,,,,,,,,,,,,
0,3.000000,3.000000,3.000000,3,3.14286,3.28571,4.57143,3.71429,3,3.14286,3.85714,3,3.28571
1,3.779763,3.312269,3.075193,2.14286,3.14286,2.71429,5.42857,4.14286,3,3.42857,4.28571,3,3.14286
2,4.762203,3.657041,3.152270,3,3.14286,2.57143,7.42857,4.85714,3,3.71429,5,3,3.71429
3,6.000000,4.037701,3.231279,2.85714,3.14286,3.42857,7.57143,5.28571,3,3.71429,4.42857,3,3.85714
4,7.559526,4.457983,3.312269,2.14286,4,3.57143,8.28571,5.85714,4.14286,5,4.57143,3.42857,4.57143
5,9.524406,4.922012,3.395288,4,2.57143,2.85714,10.4286,6,4.14286,5.14286,4.28571,3.85714,4.42857
6,12.000000,5.434342,3.480388,3.85714,2.28571,3.14286,12.1429,6.85714,4.14286,5.28571,4,4.57143,4.42857
7,15.119053,6.000000,3.567621,3.71429,1.28571,2.28571,16,6,2.14286,5.85714,3.85714,2.57143,3.28571
8,19.048813,6.624537,3.657041,3.71429,1.28571,2.42857,19.4286,5.42857,2.14286,6.14286,4,3.14286,4.42857
